# Введение в глубинное обучение, ФКН ВШЭ

## Домашнее задание 1. Введение в PyTorch. Полносвязные нейронные сети.

### Общая информация

Дата выдачи: 05.10.2021

Мягкий дедлайн: 23:59MSK 19.10.2021

Жесткий дедлайн: 23:59MSK 23.10.2021

Оценка после штрафа после мягкого дедлайна вычисляется по формуле $M_{penalty} = M_{full} \cdot 0.85^{t/1440}$, где $M_{full}$ &mdash; полная оценка за работу без учета штрафа, а $t$ &mdash; время в минутах, прошедшее после мягкого дедлайна (округление до двух цифр после запятой). Таким образом, спустя первые сутки после мягкого дедлайна вы не можете получить оценку выше **8.5**, а если сдать перед самым жестким дедлайном, то ваш максимум &mdash; **5.22** балла.

### Оценивание и штрафы
Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.
Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В этом задании вам предстоит предсказывать год выпуска песни по некоторым звуковым признакам: [данные](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [1]:
import pandas as pd
import numpy as np
import random

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm
from IPython.display import display, clear_output

In [2]:
# !wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

--2021-10-10 10:22:43--  https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Распознаётся archive.ics.uci.edu (archive.ics.uci.edu)… 128.195.10.252
Подключение к archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 211011981 (201M) [application/x-httpd-php]
Сохранение в: «data.txt.zip»

data.txt.zip        100%[===================>] 201,24M   541KB/s    за 13m 7s  

2021-10-10 10:35:51 (262 KB/s) - «data.txt.zip» сохранён [211011981/211011981]



In [2]:
df = pd.read_csv('data.txt.zip', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [3]:
df = df.astype('float32')

In [4]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

In [5]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

ss2 = StandardScaler()
y_train = ss2.fit_transform(y_train[:, None]).reshape(-1)
y_test = ss2.transform(y_test[:, None]).reshape(-1)

## Задание 0. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

In [6]:
model = Ridge()
model = model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred = y_pred * ss2.scale_ + ss2.mean_
y_target = y_test * ss2.scale_ + ss2.mean_
y_train_rescaled = y_train * ss2.scale_ + ss2.mean_

print("Test RMSE = %.4f" % mean_squared_error(y_target, y_pred, squared=False))

Test RMSE = 9.5102


In [7]:
best_constant = y_train_rescaled.mean()
print("Test RMSE with best constant = %.4f" % mean_squared_error(y_target, best_constant * np.ones(y_test.shape),
                                                                 squared=False))
print("Train RMSE with best constant = %.4f" % mean_squared_error(y_train_rescaled, best_constant * np.ones(y_train.shape),
                                                                  squared=False))

Test RMSE with best constant = 10.8525
Train RMSE with best constant = 10.9398


## Задание 1. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знайте, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из ячейки ниже.



In [8]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [149]:
set_random_seed(42)

Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться следующими сигнатурами функций. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [150]:
class MyDataset(Dataset):
    def __init__(self, X, y):
        super().__init__()
        self.X = X
        self.y = y
        self.len = len(X)

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
    
class Model(nn.Module):
    def __init__(self, in_features = 90, out_features = 1):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features

        self.sequential = nn.Sequential(
            nn.Linear(in_features, 90),
            nn.ReLU(),
            nn.Linear(90, 60),
            nn.BatchNorm1d(60),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(60, 30),
            nn.BatchNorm1d(30),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(30, 9),
            nn.ReLU(),
            nn.Linear(9, self.out_features)
        )

    def forward(self, x):
        x = self.sequential(x)
        return x


class TrainTest():
    def __init__(self, X_train, y_train, X_test, y_test, params={}):
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.params = params
        
        self.set_constants()
        self.set_model()
        self.load_data()
        
        
    def set_constants(self):
        self.NUM_EPOCHS = 7
        self.LR = 2e-3
        self.WEIGHT_DECAY = 1e-4
        self.GAMMA = 0.9999
        self.BATCH_SIZE = 64
        self.EVAL_BATCH_SIZE = 300
        
        if self.params != {}:
            self.NUM_EPOCHS = self.params['epochs']
            self.LR = self.params['lr']
            self.WEIGHT_DECAY = self.params['weight_decay']
            self.GAMMA = self.params['gamma']
        
    def load_data(self):
        train_dl = DataLoader(
            MyDataset(self.X_train, self.y_train),
            batch_size = self.BATCH_SIZE,
            shuffle = True
        )
        
        

        test_dl = DataLoader(
            MyDataset(X_test, y_test),
            batch_size = self.EVAL_BATCH_SIZE,
            shuffle = False
        )
        
        self.data_loader = {'train': train_dl, 'test': test_dl}
        
    def set_model(self):
        self.model = Model()
        self.criterion = nn.MSELoss()
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.LR, weight_decay=self.WEIGHT_DECAY)
        self.scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=self.GAMMA)
        
    def train_model(self, model, optimizer, criterion, scheduler, show_progress=True):
        metrics_dict = {
            "Epoch": [],
            "Train RMSE": [],
            "Test RMSE": [],
        }

        for epoch in range(self.NUM_EPOCHS):
            metrics_dict["Epoch"].append(epoch)
            for stage in ['train', 'test']:
                with torch.set_grad_enabled(stage == 'train'):
                    if stage == 'train':
                        model.train()
                    else:
                        model.eval()
                        
                    loss_at_stage = 0
                    for batch in self.data_loader[stage]:
                        x_batch, y_batch = batch
                        
                        y_pred = model(x_batch).view(-1)
                        loss = criterion(y_pred, y_batch)
                        if stage == "train":
                            loss.backward()
                            optimizer.step()
                            scheduler.step()
                            optimizer.zero_grad()
                    
                        with torch.no_grad():
                            loss_at_stage += (torch.square((y_pred - y_batch)).sum()).item()
                            
                        
                    rmse_at_stage = (loss_at_stage / len(self.data_loader[stage].dataset)) ** (1/2)
                    metrics_dict[f"{stage.title()} RMSE"].append(rmse_at_stage)
            if show_progress:
                clear_output(wait=True)
                display(pd.DataFrame(metrics_dict))
        self.model = model
        
    
    def test(self):
        with torch.no_grad():
            preds = self.model(torch.Tensor(self.X_test)).view(-1)
        self.preds_rescheduled = preds * ss2.scale_ + ss2.mean_
        self.y_test_rescheduled = self.y_test * ss2.scale_ + ss2.mean_
        return self.preds_rescheduled, self.y_test_rescheduled
    

In [151]:
def calc_rmse(preds, y_true):
    return ((preds - y_true) ** 2).mean() ** (1/2)

## Fit HP

In [152]:
def fit_hp(gammas, weight_decays, learning_rate, epochs, show_progress):
    params = pd.DataFrame({'gamma': [], 'lr': [], 'weight': [], 'error': []})

    id_ = 0
    for gamma in tqdm(gammas):
        for weight in weight_decays:
            for lr in learning_rate:
                set_random_seed(42)
                
                model = TrainTest(X_train, y_train,
                                  X_test, y_test, 
                                  params={'gamma': gamma,
                                          'weight_decay': weight,
                                          'lr': lr,
                                          'epochs': epochs
                                         }
                                 )
                model.train_model(model.model, model.optimizer, model.criterion, model.scheduler, show_progress)
                y_preds, y_test_ = model.test()
                error = calc_rmse(y_preds, y_test_)

                params.loc[id_] = [gamma, lr, weight, error]
                id_ += 1
                
    return params

In [125]:
gammas = [0.9995, 0.9997, 0.9999, 0.99993]
weight_decays = [1e-4, 1e-3, 2e-3]
learning_rate = [2e-3, 4e-3, 1e-2, 3e-2, 4e-2, 1e-1]
params = fit_hp(gammas, weight_decays, learning_rate, epochs=3, show_progress=False)


  0%|          | 0/4 [00:00<?, ?it/s]

In [128]:
import plotly.express as px

fig = px.line(params, y="error", hover_data=['gamma', 'lr', 'weight'], title='Fit HP')

fig.show()

In [153]:
best_params = params.sort_values('error', ascending=True).head(1)
best_gamma = float(best_params['gamma'])
best_weight_decay = float(best_params['weight'])
best_lr = float(best_params['lr'])

In [154]:
model = TrainTest(X_train, y_train, X_test, y_test, 
                  params={'gamma': best_gamma,
                          'weight_decay': best_weight_decay,
                          'lr': best_lr,
                          'epochs': 7
                         }
                 )
model.train_model(model.model, model.optimizer, model.criterion, model.scheduler)
y_preds_rescaled, y_test_rescaled = model.test()

,Epoch,Train RMSE,Test RMSE
0,0,0.822635,0.811459
1,1,0.800092,0.802389
2,2,0.791152,0.802143
3,3,0.785839,0.799222
4,4,0.782755,0.800074
5,5,0.780969,0.800485
6,6,0.780593,0.799374


In [155]:
print("RMSE: %.3f" % calc_rmse(y_preds_rescaled, y_test_rescaled).item())

RMSE: 8.745


In [25]:
assert y_preds_rescaled.shape[0] == y_test.shape[0]

## Задание 2. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

## Not big otchet

Работа выполнялась с использованием следующих библиотек: __pytorch__ (для работы с нейронной сетью), __pandas & numpy__ (для работы с данными), __plotly__ (построение графиков).

Изначально была обучена простейшая модель с двумя линейными слоями и одним нелинейным слоем (__ReLU__), RMSE на тесте была аналогична ошибке при обучении __Ridge__ модели (около __9.5__).

В качестве следующего этапа я добавил больше комбинаций линейных и нелинейных слоев, оптимальным значением количества линейных слоев оказалось __5__. Количество нейронов в линейных слоях старался уменьшать примерно в __k__ раз с каждым добавлением нового линейного слоя  и мне удалось получить RMSE ≈ __9__

Для уменьшения ошибки я начал использовать дополнительные слои __Dropout & BatchNorm__, в качестве параметра для __Dropout__ я использовал __p=0.2__, это позволило уменьшить ошибку до __8.8__.


В качестве оптимизационного алгоритма я использовал __AdamW__.

Для дальнейшей оптимизации я добавил в свою модель экспоненциальное уменьшение __learning_rate__ модели с помщью __lr_scheduler.ExponentialLR__.

Для достижения финальной ошибки был организован перебор следующих гиперпараметров модели: __gamma__ (scheduler) и __learning rate, weight_decay__ (optimizer).